In [ ]:
import os
from athina.evals import RagasAnswerCorrectness, RagasAnswerRelevancy, RagasContextRelevancy, RagasFaithfulness 
from athina.loaders import RagasLoader
from athina.keys import AthinaApiKey, OpenAiApiKey
from athina.runner.run import EvalRunner
from athina.datasets import yc_query_mini
from athina.interfaces.athina import AthinaExperiment
import pandas as pd

OpenAiApiKey.set_key(os.getenv('OPENAI_API_KEY'))
AthinaApiKey.set_key(os.getenv('ATHINA_API_KEY'))

In [ ]:
# Create batch dataset from list of dict objects
raw_data = yc_query_mini.data

dataset = RagasLoader().load_dict(raw_data)
pd.DataFrame(dataset)

In [ ]:
# Run the eval suite
eval_model = "gpt-4-1106-preview"
eval_suite = [
    RagasAnswerCorrectness(),
    RagasFaithfulness(),
    RagasContextRelevancy(),
    RagasAnswerRelevancy(),
]

# Run the evaluation suite
batch_eval_result = EvalRunner.run_suite(
    evals=eval_suite,
    data=dataset,
    max_parallel_evals=1,   # If you increase this, you may run into rate limits
)

pd.DataFrame(batch_eval_result)